# 데이터 전처리 방법 1
### 1. 누락된 데이터 다루기  
     1-1. 테이블 형태에 데이터에서 누락된 값 식별하는 방법  
     1-2. 누락된 값이 있는 데이터 삭제하는 방법  
     1-3. 누락된 데이터 대체하는 방법  


### 2. 범주형 데이터 다루기  
     2-1. 순서가 있는 데이터 매핑  
     2-2. 클래스 레이블 매핑  
     2-3. 순서가 없는 데이터 매핑  

## 1. 누락된 데이터 다루기

### 1-1. 테이블 형태 데이터에서 누락된 값 식별

In [4]:
import pandas as pd
from io import StringIO
import sys

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


"StringIO" 클래스는 문자열을 파일처럼 다룰 수 있게 해주는 클래스이다.  
csv 파일을 만들지 않고 csv 형태로 문자열을 만듬으로써 csv파일을 사용할 수 있게 되었다.  
2번째와 3번째 행에 누락된 데이터가 있는 데이터 프레임을 만들었다.

In [5]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [6]:
df.isnull().sum(axis=1) # 각 행에 null의 합계
df.isnull().sum()       # 각 열에 null의 합계   

A    0
B    0
C    1
D    1
dtype: int64

각 열에 null 값의 합계를 보여주고 만약에 axis =1 을 파라미터로 주게 되면 각 행에 합계를 보여주게 된다.

In [7]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

df.values를 사용함으로써 df 파일을 numpy 배열로 만들 수 있다.

### 1-2. 누락된 값이 있는 훈련 샘플이나 특성 제외

In [8]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


dropna(axis=0)는 누락된 값이 있는 **행**을 삭제한다.  
따라서 null 값이 없는 0번 째 행만 살아남았다.

In [9]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


dropna(axis=1)는 누락된 값이 있는 **열**을 삭제한다.  
따라서 null 값이 없는 A와 B의 열만 살아남았다.

In [10]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


dropna(how='all')은 모든 열이 nan인 행을 삭제한다.  
기본값은 how='any'이다.

In [11]:
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


dropna(thresh=4)은 nan을 제외한 열의 값의 수가 4개보다 작은 행을 삭제한다.

In [12]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


dropna(subset=['C'])는 C열에 nan이 있는 행을 삭제한다.

### 1-3. 누락된 값 대체

In [13]:
#원래 배열
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [14]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
#imr = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=1)
#imr = SimpleImputer(missing_values=np.nan, strategy='median')
#imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

열의 평균으로 nan 값을 채우는 기능이다.  
3과 12의 **평균값** 7.5  
4와 8의 **평귭값** 6으로 nan 값이 채워진걸 볼 수 있다.  

strategy에 매개변수에는 mean, median, most_frequent, constant(fill_value)가 있다.

In [15]:
from sklearn.preprocessing import FunctionTransformer

ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

행의 평균으로 nan값을 채우고 싶다면 'FunctionTransformer'클래스라는 특정 함수를 변환기처럼 사용해서 각 샘플에 적용해주는 클래스를 사용한다.  
fit_transform은 fit과 transform함수를 동시에 사용 가능한 함수이다.

In [16]:
imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ,  0. ,  0. ],
       [ 5. ,  6. ,  7.5,  8. ,  1. ,  0. ],
       [10. , 11. , 12. ,  6. ,  0. ,  1. ]])

In [17]:
imr.indicator_

MissingIndicator(error_on_new=False)

In [18]:
imr.indicator_.features_

array([2, 3], dtype=int64)

In [19]:
imr.indicator_.fit_transform(df.values)

array([[False, False],
       [ True, False],
       [False,  True]])

In [20]:
#원래 데이터를 확인 할 수 있다.
imr.inverse_transform(imputed_data)

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

#### 누락된 값 예측하는 방법

<font color=red>simpleImputer</font>는 한 특성의 통계값을 사용하여 누락된 값을 채우는 반면 <font color=red>
IterativeImputer</font> 클래스는 다른 특성을 사용하여 누락된 값을 예측 한다.
***
먼저 <font color=red>initail_strategy</font> 매개변수에 지정된 방식으로 누락된 값을 초기화한 후 그 다음 누락된 값이 있는 한 특성을 타깃으로 삼고 다른 특성을 사용해 모델을 훈련하여 예측한다.  
+ initail_strategy 매개변수에 지정할 수 있는 값은 SimpleImputer와 동일하게 <font style="background-color:yellow;">mean</font>,
<font style="background-color:yellow;">median</font>,<font style="background-color:yellow;">most_frequent</font>,
<font style="background-color:yellow;">constant</font>가 가능하다.  
***
예측할 특성을 선택하는 방법에는
+ ascending: 누락된 값이 가장 적은 특성부터 선택(기본값)
+ descending: 누락된 값이 가장 큰 특성부터 선택
+ roman: 왼쪽에서 오른쪽으로 선택
+ arabic: 오른쪽에서 왼쪽으로 선택
+ random: 랜덤 선택
가 있다.

특정 예측은 종료 조건을 만족할 때까지 반복한다.
* 각 반복 단계에서 이전 단계와 절댓값 차이 중 가장 큰 값이 누락된 값을 제외하고 가장 큰 절대값에 tol 매개 변수를 곱한 것보다 작을 경우 종류, tol 매개변수의 기본값은 1e-3
* max_iter 매개변수에서 지정한 횟수에 도달할 때 종류, max_iter의 기본값은 10

예측에 사용하는 모델은 estimator 매개변수에서 지정할 수 있으며, 기본적으로 BayesianRidge 클래스를 사용한다.
예측에 사용할 특정 개수는 n_nearest_features에서 지정할 수 있으며 상관계수가 높은 특성을 우선하여 랜덤하게 선택한다. 기본값은 None

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

In [22]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [23]:
df.fillna(method='bfill')
#df.fillna(method='bfill', axis=1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


fillna(method='bfill')은 fillna(method='backfill')와 같으며 nan의 다음 행의 값으로 nan 값을 채우는 방법이다.  
이때 axis=1을 지정하면 다음 열의 값으로 nan값을 채우게 된다.

In [24]:
df.fillna(method='ffill')
#df.fillna(method='ffill', axis=1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


fillna(method='ffill')은 fillna(method='pad')와 같은 방법으로 nan의 이전 값으로 nan값을 채우게 된다.  
이때 axis=1을 지정하면 이전 열의 값으로 nan값을 채우게 된다.

***

## 2. 범주형 데이터 다루기

판다스를 사용한 범주형 데이터 인코딩

In [25]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 2-1. 순서가 있는 특성 매핑

In [26]:
size_mapping = {'XL':3,
               'L':2,
               'M':1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [27]:
#다시 되돌리는 방법
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 2-2. 클래스 레이블 인코딩

In [28]:
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

클래스 레이블을 문자열에서 정수로 바꾸기 위해 매핑 딕셔너리를 만든다.

In [29]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [30]:
#클래스 레이블을 거꾸로 매핑한다.
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


#### 사이킷런의 LabelEncoder을 사용하여 레이블 인코딩이 가능하다

In [31]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [32]:
#거꾸로 매핑
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

In [33]:
class_le.classes_

array(['class1', 'class2'], dtype=object)

### 2-3. 순서가 없는 특성에 원-핫 인코딩 적용

In [34]:
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

위에 방법도 사용 가능하지만 sklearn에 OrdinalEncoder와 ColumnTransformer을 사용하는 방법도 있다.

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder(dtype = int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

In [36]:
#거꾸로 인코딩
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

#### 순서가 없는 특성일 경우 "OneHotEncoder"를 사용한다.

In [37]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:,0].reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [41]:
from sklearn.compose import ColumnTransformer

X = df[['color','size','price']].values
c_transf= ColumnTransformer([('onehot', OneHotEncoder(), [0]),
                            ('nothing', 'passthrough', [1,2])])
c_transf.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

0번째 컬럼인 color 컬럼에만 onehotencoder를 적용하고 다른 컬럼에는 적용하지 말라는 조건이다.  
만약 정수로 인코딩하고 싶다면 OneHotEncoder()함수에 dtype=np.int로 파라미터값을 주면 된다.

In [43]:
#판다스에서 제공되는 one-hot 인코딩
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [44]:
#column 매개변수를 사용하여 변환하려는 특성을 구체적으로 지정할 수 있다.
pd.get_dummies(df[['price','color','size']], columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,1,0,0
1,13.5,red,0,1,0
2,15.3,blue,0,0,1


<font style="color:blue;">순서가 있는 특성 인코딩</font>

순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신이 없거나 두 번주 사이의 순서를 정의할 수 었다면 임계값을 사용해 0이나 1로 인코딩 할 수있다.

In [46]:
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


판다스 데이터 프레임의 apply 메서드를 사용해 임계값 기준으로 특성을 인코딩하는 lambda 함수를 적용할 수 있다.

In [47]:
df['x > M'] = df['size'].apply(lambda x: 1 if x in {'L', 'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x: 1 if x == 'XL' else 0)

del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1
